In [28]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('deli.csv')
df2 = pd.read_csv('mt.csv')


In [29]:
total_scoredf = df1.groupby(['match_id', "inning"]).sum()['total_runs'].reset_index()

In [30]:
total_scoredf = total_scoredf[total_scoredf["inning"] == 1]

In [32]:
match_df = df2.merge(total_scoredf[['match_id', 'total_runs']], left_on = 'id', right_on = 'match_id')


In [33]:
match_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [34]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [35]:
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]
match_df = match_df[match_df['dl_applied'] == 0]

In [36]:
match_df = match_df[['match_id','city', 'winner','total_runs']]

In [37]:
delivery_df = df1[df1['inning'] == 2]

In [38]:
delivery_df = match_df.merge(delivery_df , on = 'match_id')

In [39]:
delivery_df['current_score'] = delivery_df.groupby('match_id')['total_runs_y'].cumsum()


In [40]:
delivery_df['runs_left']  = delivery_df['total_runs_x'] - delivery_df['current_score']
delivery_df['balls left'] = 126 - delivery_df['over']*6 - delivery_df['ball']
delivery_df.head(4)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls left
0,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,1,0,1,NaN,NaN,NaN,1,206,119
1,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,NaN,NaN,NaN,1,206,118
2,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,NaN,NaN,NaN,1,206,117
3,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,2,0,2,NaN,NaN,NaN,3,204,116


In [41]:
# Step 1: Fill NaN values with "0"
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna(0)
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x: 0 if x == 0 else 1)




In [42]:
delivery_df['wickets'] = delivery_df.groupby('match_id')['player_dismissed'].cumsum()
delivery_df['wickets_left'] = 10 - delivery_df['wickets']
delivery_df['crr'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls left'])
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls left']

In [43]:
def result(row) :
    if row['batting_team'] == row['winner']:
      return 1
    else: return 0

In [44]:
delivery_df['result'] = delivery_df.apply(result, axis = 1)

In [45]:
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls left','wickets','total_runs_x','crr','rrr','result']]

In [46]:
final_df.dropna(inplace=True)
final_df = final_df[final_df['balls left'] != 0]



C:\Users\bokde\AppData\Local\Temp\ipykernel_15992\2628493673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(inplace=True)


In [47]:
x = final_df.drop('result', axis = 1)
y = final_df['result']

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state = 43)


In [49]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features =['batting_team','bowling_team', 'city']

One_hot = OneHotEncoder()
transformer = ColumnTransformer([('One_hot',One_hot, categorical_features)],remainder = "passthrough")
X = transformer.fit_transform(X_train)
x_test = transformer.fit_transform(X_test)

In [53]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X, y_train)


RandomForestClassifier()

In [51]:
model.score(x_test, y_test)

0.9984581960894247